In [5]:
%%configure -f

{"name": "Derek", "driverMemory": "8G",  "executorMemory": "3G", "executorCores": 2, "numExecutors": 100, "conf": { "spark.sql.parquet.writeLegacyFormat": true}} 

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4237,application_1571711215089_3117,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4231,application_1571711215089_3111,shared,dead,Link,,
4237,application_1571711215089_3117,pyspark,idle,Link,Link,✔


# Modified queries

## 1. bl_sales_sku_st_wk

In [17]:
spark.sql("""
select
 main.product_key
,main.store_key
,main.fiscal_year
,main.fiscal_month
,main.fiscal_week_of_year
,main.fiscal_year_week
,d.fiscal_week_start_date as period_start_date
,d.fiscal_week_end_date as period_end_date
,case when d.fiscal_week_of_year is null then 0 else 1 end 	as valid_date_flag ---to take care week 53 problem
,main.max_fiscal_date
,d.vs_max_sales_year -- if it is invalid week 53, it will show null
,d.vs_max_sales_month -- if it is invalid week 53, it will show null
,d.vs_max_sales_week -- if it is invalid week 53, it will show null
,bu.bu_key 													as bu_code
,bu.bu_name
,p.principle_supplier_code 									as principle_supplier_code
,p.mother_company_name 										as mother_company_name 
,p.brand_type_name
,p.brand_name
,nvl(p.ext_h9_l1_hierarchy_code,'-') 						as h9_l1_product_hierarchy_code
,nvl(substring(p.ext_h9_l1_hierarchy_code,3,2),'-') 		as h9_l1_product_no
,p.ext_h9_l1_hierarchy_name 								as h9_l1_product_hierarchy_name
,nvl(p.ext_h9_l2_hierarchy_code,'-') 						as h9_l2_product_hierarchy_code
,nvl(substring(p.ext_h9_l2_hierarchy_code,3,2),'-') 		as h9_l2_product_no
,p.ext_h9_l2_hierarchy_name 								as h9_l2_product_hierarchy_name
,nvl(p.ext_h9_l3_hierarchy_code,'-') 						as h9_l3_product_hierarchy_code
,nvl(substring(p.ext_h9_l3_hierarchy_code,3,2),'-') 		as h9_l3_product_no
,p.ext_h9_l3_hierarchy_name									as h9_l3_product_hierarchy_name
,nvl(p.ext_h9_l4_hierarchy_code,'-')						as h9_l4_product_hierarchy_code
,nvl(substring(p.ext_h9_l4_hierarchy_code,3,2),'-')			as h9_l4_product_no
,p.ext_h9_l4_hierarchy_name					 				as h9_l4_product_hierarchy_name
,p.product_id
,p.product_other_name 										as product_name_eng
,p.product_name
,case when nvl(substring(p.h1_l1_hierarchy_code,3,2),'-') = '09' then p.h1_l1_hierarchy_name
when nvl(substring(p.h1_l2_hierarchy_code,7,2),'-')>94 then p.h1_l2_hierarchy_name
when nvl(substring(p.h1_l2_hierarchy_code,7,2),'-')=90 then p.h1_l2_hierarchy_name
when p.product_id in ('989898','979797','666666') then p.product_other_description else 1 end as sku_flag
,s.store_id
,s.store_code
,s.store_name
,s.store_other_name
,s.store_format_code
,s.store_format_name
,s.web_store_flag
,s.comp_store_flag
,s.h1_l2_hierarchy_name 									as Region
,s.h1_l5_hierarchy_name 									as Area
,SUBSTRING(s.address_5,1,3) 								as State
,SUBSTRING(s.city_name,4,5)                                 as City_Name
,case when s.web_store_flag = 'Y' then 1 Else 0 END			as store_type
,case when s.web_store_flag = 'Y' then 'Online Store' 
	else 'Physical Store' END 								as store_type_description
,concat(s.address_1, s.address_2) 							as Loc_Addr
,cast(s.latitude AS STRING)									as store_latitude
,cast(s.longitude AS STRING)								as store_longitude
,nvl(main.net_sales_amt,0) 									as net_sales_amt
,nvl(main.cogs_amt,0) 										as cogs_amt
,nvl(main.gross_profit,0) 									as gross_profit
,nvl(main.gross_margin,0) 									as gross_margin
,nvl(main.net_sales_adj_amt,0) 								as net_sales_adj_amt
,nvl(main.compensation,0) 									as compensation
,nvl(main.sales_qty,0) 										as sales_qty
,nvl(main.net_sales_shrinkage,0) 							as net_sales_shrinkage
,nvl(main.cogs_shrinkage,0) 								as cogs_shrinkage
,nvl(main.sales_qty_shrinkage,0) 							as sales_qty_shrinkage
,nvl(main.net_sales_amt_ly,0) 								as net_sales_amt_ly
,nvl(main.cogs_amt_ly,0) 									as cogs_amt_ly
,nvl(main.gross_profit_ly,0) 								as gross_profit_ly
,nvl(main.gross_margin_ly,0) 								as gross_margin_ly
,nvl(main.net_sales_adj_amt_ly,0) 							as net_sales_adj_amt_ly
,nvl(main.compensation_ly,0) 								as compensation_ly
,nvl(main.sales_qty_ly,0)									as sales_qty_ly
,nvl(main.net_sales_shrinkage_ly,0) 						as net_sales_shrinkage_ly
,nvl(main.cogs_shrinkage_ly,0) 								as cogs_shrinkage_ly
,nvl(main.sales_qty_shrinkage_ly,0) 						as sales_qty_shrinkage_ly
,current_timestamp as table_last_update
from (
        select 
             nvl(ty.product_key,ly.product_key) 							as product_key
            ,nvl(ty.store_key,ly.store_key) 								as store_key
			,nvl(ty.bu_key,ly.bu_key) 										as bu_key
            ,nvl(ty.fiscal_year,cast(ly.fiscal_year+1 as INT)) 				as fiscal_year
            ,nvl(ty.fiscal_month,ly.fiscal_month) 							as fiscal_month
            ,nvl(ty.fiscal_week_of_year,ly.fiscal_week_of_year) 			as fiscal_week_of_year
            ,nvl(ty.fiscal_year_week,cast(ly.fiscal_year_week+100 as INT)) 	as fiscal_year_week --this will generate week 53
            ,nvl(ty.max_fiscal_date,ly.max_fiscal_date) 					as max_fiscal_date
            ,nvl(ty.net_sales_amt,0) 										as net_sales_amt
            ,nvl(ty.cogs_amt,0) 											as cogs_amt
            ,nvl(ty.gross_profit,0) 										as gross_profit
            ,nvl(ty.gross_profit,0)+nvl(ty.compensation,0)					as gross_margin
            ,nvl(ty.net_sales_adj_amt,0) 									as net_sales_adj_amt
			,nvl(ty.compensation,0) 										as compensation
            ,nvl(ty.sales_qty,0) 											as sales_qty
            ,nvl(ty.net_sales_shrinkage,0) 									as net_sales_shrinkage
            ,nvl(ty.cogs_shrinkage,0) 										as cogs_shrinkage
            ,nvl(ty.sales_qty_shrinkage,0) 									as sales_qty_shrinkage
            ,nvl(ly.net_sales_amt,0) 										as net_sales_amt_ly
            ,nvl(ly.cogs_amt,0)												as cogs_amt_ly
            ,nvl(ly.gross_profit,0) 										as gross_profit_ly
            ,nvl(ly.gross_profit,0)+nvl(ly.compensation,0)					as gross_margin_ly
            ,nvl(ly.net_sales_adj_amt,0) 									as net_sales_adj_amt_ly
			,nvl(ly.compensation,0) 										as compensation_ly
            ,nvl(ly.sales_qty,0) 											as sales_qty_ly
            ,nvl(ly.net_sales_shrinkage,0) 									as net_sales_shrinkage_ly
            ,nvl(ly.cogs_shrinkage,0) 										as cogs_shrinkage_ly
            ,nvl(ly.sales_qty_shrinkage,0) 									as sales_qty_shrinkage_ly
        from aggregates.temp_sales_aggregate_wk ty
        full outer join aggregates.temp_sales_aggregate_wk ly on ty.product_key = ly.product_key
                                                          and ty.store_key = ly.store_key
                                                          and ty.fiscal_year -1 = ly.fiscal_year 
                                                          and ty.fiscal_week_of_year = ly.fiscal_week_of_year
        where ty.vs_max_sales_year >=-2 or ly.vs_max_sales_year in (-1,-2,-3)
    ) main
Left join aggregates.product p 			on p.product_key 		= main.product_key
Left join aggregates.store s 				on s.store_key 			= main.store_key
left join (
            select fiscal_year,fiscal_week_of_year,fiscal_week_start_date,fiscal_week_end_date,vs_max_sales_year,vs_max_sales_month,vs_max_sales_week from aggregates.bl_business_date d
            group by fiscal_year,fiscal_week_of_year,fiscal_week_start_date,fiscal_week_end_date,vs_max_sales_year,vs_max_sales_month,vs_max_sales_week
            ) d on d.fiscal_week_of_year = main.fiscal_week_of_year
				and d.fiscal_year 		= main.fiscal_year
left join rdm.d_business_unit bu 	on bu.bu_key = main.bu_key
where 
(d.vs_max_sales_week <= - 1
or (date_format(max_fiscal_date, 'EEEE') = 'Sunday' and d.vs_max_sales_week <= 0))

""").write.mode("overwrite").saveAsTable("develop.bl_sales_sku_st_wk")

## 2. bl_sales_brd_h9l2_st_sup_wk

In [ ]:
spark.sql("""

SELECT
area
, brand_name
, brand_type_name
, bu_code
, bu_name
, h9_l1_product_hierarchy_code
, h9_l1_product_no
, h9_l1_product_hierarchy_name
, h9_l2_product_hierarchy_code
, h9_l2_product_no
, h9_l2_product_hierarchy_name
, comp_store_flag
, fiscal_month
, fiscal_week_of_year
, fiscal_year
, fiscal_year_week
, max_fiscal_date
, vs_max_Sales_Month
, vs_max_Sales_Week
, vs_max_Sales_Year
, period_end_date
, period_start_date
, region
, state
, city_name
, store_key
, store_id
, store_code
, store_name
, store_other_name
, store_format_code
, store_format_name
, store_type_description
, store_latitude
, store_longitude
, principle_supplier_code
, mother_company_name 
, web_store_flag
, store_type
, valid_date_flag
, sku_flag
, SUM(net_sales_amt) 				AS net_sales_amt
, SUM(net_sales_amt_ly)			AS net_sales_amt_ly
, SUM(gross_profit) 		AS gross_profit
, SUM(gross_profit_ly) 	AS gross_profit_ly
, SUM(gross_margin) 		AS gross_margin
, SUM(gross_margin_ly) 	AS gross_margin_ly
, SUM(cogs_amt) 				AS cogs_amt
, SUM(cogs_amt_ly) 			AS cogs_amt_ly
, SUM(net_sales_shrinkage) 		AS net_sales_shrinkage
, SUM(net_sales_shrinkage_ly) 	AS net_sales_shrinkage_ly
, SUM(sales_qty_shrinkage) 					AS sales_qty_shrinkage
, SUM(sales_qty_shrinkage_ly) 				AS sales_qty_shrinkage_ly
, SUM(net_sales_adj_amt) 				AS net_sales_adj_amt
, SUM(net_sales_adj_amt_ly) 			AS net_sales_adj_amt_ly
, SUM(cogs_shrinkage) 		AS cogs_shrinkage
, SUM(cogs_shrinkage_ly) 	AS cogs_shrinkage_ly
, SUM(compensation) 				AS compensation
, SUM(compensation_ly) 				AS compensation_ly
, SUM(sales_qty) 							AS sales_qty
, SUM(sales_qty_ly) 						AS sales_qty_ly
,current_timestamp as table_last_update
FROM develop.bl_sales_sku_st_wk
GROUP BY 
area
, brand_name
, brand_type_name
, bu_code
, bu_name
, h9_l1_product_hierarchy_code
, h9_l1_product_no
, h9_l1_product_hierarchy_name
, h9_l2_product_hierarchy_code
, h9_l2_product_no
, h9_l2_product_hierarchy_name
, comp_store_flag
, fiscal_month
, fiscal_week_of_year
, fiscal_year
, fiscal_year_week
, max_fiscal_date
, vs_max_Sales_Month
, vs_max_Sales_Week
, vs_max_Sales_Year
, period_end_date
, period_start_date
, region
, state
, city_name
, store_key
, store_id
, store_code
, store_name
, store_other_name
, store_format_code
, store_format_name
, store_type_description
, store_latitude
, store_longitude
, principle_supplier_code
, mother_company_name 
, web_store_flag
, store_type
, valid_date_flag
, sku_flag


""").write.mode("overwrite").saveAsTable("develop.bl_sales_brd_h9l2_st_sup_wk")

## 3. bl_sales_brd_h9l4_st_sup_wk

In [ ]:
spark.sql("""	
SELECT
area
, brand_name
, brand_type_name
, bu_code
, bu_name
, h9_l1_product_hierarchy_code
, h9_l1_product_no
, h9_l1_product_hierarchy_name
, h9_l2_product_hierarchy_code
, h9_l2_product_no
, h9_l2_product_hierarchy_name
, h9_l3_product_hierarchy_code
, h9_l3_product_no
, h9_l3_product_hierarchy_name
, h9_l4_product_hierarchy_code
, h9_l4_product_no
, h9_l4_product_hierarchy_name
, comp_store_flag
, fiscal_month
, fiscal_week_of_year
, fiscal_year
, fiscal_year_week
, max_fiscal_date
, vs_max_sales_month
, vs_max_sales_week
, vs_max_sales_year
, period_end_date
, period_start_date
, region
, state
, city_name
, store_key
, store_id
, store_code
, store_name
, store_other_name
, store_format_code
, store_format_name
, store_type_description
, store_latitude
, store_longitude
, principle_supplier_code
, mother_company_name 
, web_store_flag
, store_type
, valid_date_flag
, sku_flag
, SUM(net_sales_amt) 				AS net_sales_amt
, SUM(net_sales_amt_ly) 			AS net_sales_amt_ly
, SUM(gross_profit) 				AS gross_profit
, SUM(gross_profit_ly)				AS gross_profit_ly
, SUM(gross_margin) 				AS gross_margin
, SUM(gross_margin_ly) 				AS gross_margin_ly
, SUM(cogs_amt) 					AS cogs_amt
, SUM(cogs_amt_ly) 					AS cogs_amt_ly
, SUM(net_sales_shrinkage) 			AS net_sales_shrinkage
, SUM(net_sales_shrinkage_ly) 		AS net_sales_shrinkage_ly
, SUM(sales_qty_shrinkage) 			AS sales_qty_shrinkage
, SUM(sales_qty_shrinkage_ly) 		AS sales_qty_shrinkage_ly
, SUM(net_sales_adj_amt) 			AS net_sales_adj_amt
, SUM(net_sales_adj_amt_ly) 		AS net_sales_adj_amt_ly
, SUM(cogs_shrinkage) 				AS cogs_shrinkage
, SUM(cogs_shrinkage_ly) 			AS cogs_shrinkage_ly
, SUM(compensation) 				AS compensation
, SUM(compensation_ly) 				AS compensation_ly
, SUM(sales_qty) 					AS sales_qty
, SUM(sales_qty_ly) 				AS sales_qty_ly
, current_timestamp as table_last_update
FROM develop.bl_sales_sku_st_wk
GROUP BY 
area
, brand_name
, brand_type_name
, bu_code
, bu_name
, h9_l1_product_hierarchy_code
, h9_l1_product_no
, h9_l1_product_hierarchy_name
, h9_l2_product_hierarchy_code
, h9_l2_product_no
, h9_l2_product_hierarchy_name
, h9_l3_product_hierarchy_code
, h9_l3_product_no
, h9_l3_product_hierarchy_name
, h9_l4_product_hierarchy_code
, h9_l4_product_no
, h9_l4_product_hierarchy_name
, comp_store_flag
, fiscal_month
, fiscal_week_of_year
, fiscal_year
, fiscal_year_week
, max_fiscal_date
, vs_max_Sales_Month
, vs_max_Sales_Week
, vs_max_Sales_Year
, period_end_date
, period_start_date
, region
, state
, city_name
, store_key
, store_id
, store_code
, store_name
, store_other_name
, store_format_code
, store_format_name
, store_type_description
, store_latitude
, store_longitude
, principle_supplier_code
, mother_company_name 
, web_store_flag
, store_type
, valid_date_flag
, sku_flag

""").write.mode("overwrite").saveAsTable("develop.bl_sales_brd_h9l4_st_sup_wk")

## 4. bl_sales_sku_st_d

In [3]:
spark.sql("""
select
 main.product_key
,main.store_key
,d.fiscal_date_key -- null means 53 week
,main.fiscal_year
,main.fiscal_month
,main.fiscal_week_of_year
,main.fiscal_year_week
,main.fiscal_day_of_year
,case when d.fiscal_date_key is null then 0 else 1 end as valid_date_flag ---to take care week 53 problem
,d.fiscal_date
,main.max_fiscal_date
,d.vs_max_sales_week
,nvl(bu.bu_key,'-')									as bu_code
,nvl(bu.bu_name,'-') 								as bu_name
,nvl(p.principle_supplier_code,'-') 				as principle_supplier_code
,nvl(p.mother_company_name,'-') 					as mother_company_name
,'-' 												as brand_code
,nvl(p.ib_brand_name,'-') 							as ib_brand_name
,nvl(substring(p.h1_l1_hierarchy_code,3,2),'-') 	as Cate_No --l1_short_code
,nvl(p.h1_l1_hierarchy_name,'-')					as Category_Name --l1_name
,nvl(substring(p.h1_l2_hierarchy_code,7,2),'-') 	as Dept_No --l2_short_code
,nvl(p.h1_l2_hierarchy_name,'-') 					as Department_Name --l2_name
,nvl(substring(p.h1_l3_hierarchy_code,11,2),'-')	as Class_No --l3_short_code
,nvl(p.h1_l3_hierarchy_name,'-')					as Class_Name --l3_name
,nvl(substring(p.h1_l4_hierarchy_code,15,2),'-')	as Sub_Class_No --l4_short_code
,nvl(p.h1_l4_hierarchy_name,'-')					as Sub_Class_Name --l4_name
,nvl(p.product_id,'-')								as product_id
,nvl(p.product_other_name,'-') 						as product_name_eng
,nvl(p.product_name,'-') 							as product_name
,nvl(s.store_code,'-') 								as store_code
,nvl(s.store_name,'-') 								as store_name
,nvl(s.store_other_name,'-') 						as store_other_name
,nvl(s.store_format_code,'-') 						as store_format_code
,nvl(s.store_format_name,'-') 						as store_format_name
,case when s.web_store_flag = 'Y' then 1
    when s.web_store_flag = 'N' then 0 Else -1 END 	as store_type
,case when s.web_store_flag = 'Y' then 'Online Store'
    when s.web_store_flag = 'N' then 'Physical Store' Else 'Unknown' END as store_type_description
,nvl(concat(s.address_1, s.address_2),'-') 			as Loc_Addr
,nvl(s.address_5,'-') 								as address_5
,nvl(s.web_store_flag,'-') 							as web_store_flag
,nvl(main.net_sales_amt,0) 							as net_sales_amt --sales_amt_local_ty
,nvl(main.cogs_amt,0) 								as cogs_amt --cogs_amt_local_ty
,nvl(main.sales_qty,0) 								as sales_qty --qty_ty
,nvl(main.net_sales_amt_ly,0) 						as net_sales_amt_ly
,nvl(main.cogs_amt_ly,0) 							as cogs_amt_ly
,nvl(main.sales_qty_ly,0) 								as sales_qty_ly
,current_timestamp as table_last_update
from (
        select
             nvl(ty.product_key,ly.product_key) 							as product_key
            ,nvl(ty.store_key,ly.store_key) 								as store_key
			,nvl(ty.bu_key,ly.bu_key) 										as bu_key
            ,nvl(ty.fiscal_year,cast(ly.fiscal_year+1 as INT)) 				as fiscal_year
            ,nvl(ty.fiscal_month,ly.fiscal_month) 							as fiscal_month
            ,nvl(ty.fiscal_week_of_year,ly.fiscal_week_of_year) 			as fiscal_week_of_year
            ,nvl(ty.fiscal_year_week,cast(ly.fiscal_year_week+100 as INT)) 	as fiscal_year_week --this will generate week 53
            ,nvl(ty.fiscal_day_of_year,ly.fiscal_day_of_year) 				as fiscal_day_of_year
            ,nvl(ty.max_fiscal_date,ly.max_fiscal_date) 					as max_fiscal_date
            ,ty.net_sales_amt 												as net_sales_amt
            ,ty.cogs_amt 													as cogs_amt
            ,ty.sales_qty 													as sales_qty
            ,ly.net_sales_amt 												as net_sales_amt_ly
            ,ly.cogs_amt													as cogs_amt_ly
            ,ly.sales_qty 													as sales_qty_ly
        from aggregates.temp_sales_aggregate_d ty
        full outer join aggregates.temp_sales_aggregate_d ly on ty.product_key = ly.product_key
													 	 and ty.store_key = ly.store_key
													 	 and ty.fiscal_year -1 = ly.fiscal_year
													 	 and ty.fiscal_day_of_year = ly.fiscal_day_of_year
		where ty.vs_max_sales_year >= -1 or ly.vs_max_sales_year in (-1,-2)
    ) main
Left join aggregates.product p 			on p.product_key 		= main.product_key
Left join aggregates.store s 				on s.store_key 			= main.store_key
left join aggregates.bl_business_date d 	on d.fiscal_day_of_year = main.fiscal_day_of_year
										and d.fiscal_year 		= main.fiscal_year
left join rdm.d_business_unit bu 	on bu.bu_key 			= main.bu_key
where 0=0
and d.vs_max_sales_day <= 0
and p.h1_l1_hierarchy_name is not null
and p.h1_l1_hierarchy_name <> 'DUMMYProducts'
""").write.mode("overwrite").saveAsTable("develop.bl_sales_sku_st_d")

# 5. bl_sales_sku_stseg_wk


In [4]:
spark.sql(""" select
     product_key
    ,fiscal_year
    ,fiscal_month
    ,fiscal_week_of_year
    ,fiscal_year_week
    ,case when sum(valid_date_flag) = 0 then 0 else 1 end as valid_week_flag
    ,bu_code
    ,bu_name
    ,principle_supplier_code
    ,mother_company_name 
    ,brand_code
    ,ib_brand_name
    ,Cate_No
    ,Category_Name
    ,Dept_No
    ,Department_Name
    ,Class_No
    ,Class_Name
    ,Sub_Class_No
    ,Sub_Class_Name
    ,product_id ---- product_code = product_id
    ,product_name_eng
    ,product_name
    ,store_format_code
    ,store_format_name
    ,store_type
    ,store_type_description
    ,web_store_flag
    ,sum(net_sales_amt) as net_sales_amt
    ,sum(cogs_amt) as cogs_amt_local
    ,sum(sales_qty) as sales_qty
    ,sum(net_sales_amt_ly) as net_sales_amt_ly
    ,sum(cogs_amt_ly) as cogs_amt_ly
    ,sum(sales_qty_ly) as sales_qty_ly
    ,current_timestamp as table_last_update
from develop.bl_sales_sku_st_d
where 
(vs_max_sales_week <= - 1
or (date_format(max_fiscal_date, 'EEEE') = 'Sunday' and vs_max_sales_week <= 0))
group by
     product_key
    ,fiscal_year
    ,fiscal_month
    ,fiscal_week_of_year
    ,fiscal_year_week
    ,bu_code
    ,bu_name
    ,principle_supplier_code
    ,mother_company_name 
    ,brand_code
    ,ib_brand_name
    ,Cate_No
    ,Category_Name
    ,Dept_No
    ,Department_Name
    ,Class_No
    ,Class_Name
    ,Sub_Class_No
    ,Sub_Class_Name
    ,product_id ---- product_code = product_id
    ,product_name_eng
    ,product_name
    ,store_format_code
    ,store_format_name
    ,store_type
    ,store_type_description
    ,web_store_flag

""").write.mode("overwrite").saveAsTable("develop.bl_sales_sku_stseg_wk")

+-----------+-----------+------------+-------------------+----------------+---------------+-------+----------------+-----------------------+-------------------+----------+------------------------------+-------+--------------+-------+-------------------+--------+-------------------+------------+------------------+----------+-----------------------------------------------+------------------------+-----------------+-----------------+----------+----------------------+--------------+-------------+--------------+---------+----------------+-----------+------------+-----------------------+
|product_key|fiscal_year|fiscal_month|fiscal_week_of_year|fiscal_year_week|valid_week_flag|bu_code|bu_name         |principle_supplier_code|mother_company_name|brand_code|ib_brand_name                 |Cate_No|Category_Name |Dept_No|Department_Name    |Class_No|Class_Name         |Sub_Class_No|Sub_Class_Name    |product_id|product_name_eng                               |product_name            |store_format_

# No need to modify
## 1. bl_stock_week_cover





In [19]:
spark.sql("""SELECT 
    NVL(sales.product_key, inv.product_key) AS product_key, 
    NVL(sales.store_key, inv.store_key) AS store_key,
    NVL(sales.sales_amt_local_w13, 0)/13 AS sales_13w_avg,
    NVL(sales.qty_w13, 0)/13 AS sales_qty_13w_avg,
    NVL(sales.cogs_amt_local_w13, 0)/13 AS cogs_13w_avg,
    NVL(inv.stock_on_hand_quantity, 0) AS stock_on_hand_qty,
    NVL(inv.in_transit_quantity, 0) AS stock_in_transition_qty,  
    NVL(inv.soh_value, 0) AS stock_on_hand_value,
	NVL(inv.in_tr_value, 0) AS stock_in_transition_value
	,current_timestamp as table_last_update
FROM (
    SELECT 
        s.product_key, s.store_key,
        SUM(s.net_sales_amt) AS sales_amt_local_w13, SUM(s.sales_qty) AS qty_w13,
        SUM(s.cogs_amt) AS cogs_amt_local_w13
    FROM aggregates.bl_sales_sku_st_d s
    INNER JOIN aggregates.bl_business_date d 
    ON d.fiscal_date = s.fiscal_date
    WHERE d.fiscal_date <= d.max_fiscal_date AND d.fiscal_date >= date_add(d.max_fiscal_date,-90)
    GROUP BY s.product_key, s.store_key) sales
FULL OUTER JOIN (
	SELECT 
	    i.product_key, i.store_key, 
        i.stock_on_hand_quantity, i.stock_on_hand_quantity*i.stock_on_hand_average_unit_cost AS soh_value, 
		i.in_transit_quantity, i.in_transit_quantity*i.stock_on_hand_average_unit_cost AS in_tr_value
	FROM rdm.f_inventory i
	INNER JOIN aggregates.bl_business_date d 
		ON i.effective_start_date_key = d.fiscal_date_key
	INNER JOIN ( 
		SELECT 
		product_key
		,store_key
		,max(effective_start_date_key) as stock_updated_date 
		FROM rdm.f_inventory
		GROUP BY product_key,store_key
		) sub
		ON i.effective_start_date_key = sub.stock_updated_date 
			 AND i.product_key= sub.product_key
			 AND i.store_key= sub.store_key
		WHERE i.stock_on_hand_quantity > 0 or i.in_transit_quantity > 0 AND i.effective_end_date_key = 99991231
                ) inv 
ON inv.product_key = sales.product_key AND inv.store_key = sales.store_key
""").show(10,False)

+-----------+---------+-------------+-----------------+------------+-----------------+-----------------------+-------------------+-------------------------+-----------------------+
|product_key|store_key|sales_13w_avg|sales_qty_13w_avg|cogs_13w_avg|stock_on_hand_qty|stock_in_transition_qty|stock_on_hand_value|stock_in_transition_value|table_last_update      |
+-----------+---------+-------------+-----------------+------------+-----------------+-----------------------+-------------------+-------------------------+-----------------------+
|100027     |64       |15.105492    |0.153846         |0.000000    |3.0000           |0.0000                 |0E-8               |0E-8                     |2019-11-15 06:54:40.631|
|100027     |89       |0.000000     |0.000000         |0.000000    |0.0000           |0.0000                 |0E-8               |0E-8                     |2019-11-15 06:54:40.631|
|100027     |157      |107.287915   |1.153846         |0.000000    |2.0000           |3.0000   

## 2. al_inv_sku_st_wk

In [18]:
spark.sql("""
SELECT 201900                                     AS fiscal_year_week, 
       201900                                     AS fiscal_month, 
       prd.supplier_name                          AS supplier_local, 
       prd.mother_company_name                    AS mother_company_name, 
       prd.brand_type_name, 
       prd.brand_name, 
       Substring(prd.ext_h9_l1_hierarchy_code, 3, 2)		AS h9_l1_product_no, 
       prd.ext_h9_l1_hierarchy_name                   		AS h9_l1_product_hierarchy_name, 
       Substring(prd.ext_h9_l2_hierarchy_code, 3, 2) 		AS h9_l2_product_no, 
       prd.ext_h9_l2_hierarchy_name                  	 	AS h9_l2_product_hierarchy_name, 
       Substring(prd.ext_h9_l3_hierarchy_code, 3, 2)	 	AS h9_l3_product_no, 
       prd.ext_h9_l3_hierarchy_name     	           	 	AS h9_l3_product_hierarchy_name, 
       prd.product_id, 
       prd.product_name, 
       org.web_store_flag, 
       org.division_type_code                     AS division_type_code, 
       org.comp_store_flag, 
       org.h1_l2_hierarchy_name                   AS region, 
       org.district_name                          AS area, 
       org.store_format_name, 
       Substring(org.address_5, 1, 3)             AS state, 
       org.store_id, 
       org.store_name, 
       swc.sales_qty_13w_avg 					  AS sales_qty_13w_avg,
       swc.cogs_13w_avg							  AS cogs_13w_avg,
       swc.stock_on_hand_qty					  AS stock_on_hand_qty,
       swc.stock_in_transition_qty 				  AS stock_in_transition_qty,
       swc.stock_on_hand_value 					  AS stock_on_hand_value,
       swc.stock_in_transition_value			  AS stock_in_transition_value,
       0                                          AS net_sales_amt, 
       0                                          AS gross_margin, 
       0                                          AS sales_qty, 
       0                                          AS net_sales_amt_ly, 
       0                                          AS gross_margin_ly, 
       0                                          AS sales_qty_ly
       ,current_timestamp as table_last_update
FROM   develop.bl_stock_week_cover swc 
       INNER JOIN aggregates.product prd 
               ON swc.product_key = prd.product_key 
       INNER JOIN aggregates.store org 
               ON swc.store_key = org.store_key 
UNION ALL 
SELECT wns.fiscal_year_week, 
       wns.fiscal_month, 
       prd.supplier_name AS supplier_local, 
       wns.mother_company_name AS mother_company_name, 
       wns.brand_type_name, 
       wns.brand_name, 
       wns.h9_l1_product_no, 
       wns.h9_l1_product_hierarchy_name, 
       wns.h9_l2_product_no, 
       wns.h9_l2_product_hierarchy_name, 
       wns.h9_l3_product_no, 
       wns.h9_l3_product_hierarchy_name, 
       wns.product_id, 
       wns.product_name, 
       wns.web_store_flag, 
       'STORE'           AS division_type_code, 
       wns.comp_store_flag, 
       wns.region, 
       wns.area, 
       wns.store_format_name, 
       wns.state, 
       wns.store_id, 
       wns.store_name, 
       0                 AS sales_qty_13w_avg, 
       0                 AS cogs_13w_avg, 
       0                 AS stock_on_hand_qty, 
       0                 AS stock_in_transition_qty, 
       0                 AS stock_on_hand_value, 
       0                 AS stock_in_transition_value, 
       wns.net_sales_amt, 
       wns.gross_margin, 
       wns.sales_qty, 
       wns.net_sales_amt_ly, 
       wns.gross_margin_ly, 
       wns.sales_qty_ly
       ,current_timestamp as table_last_update
FROM   develop.bl_sales_sku_st_wk wns 
       INNER JOIN aggregates.product prd 
               ON wns.product_key = prd.product_key 
WHERE  vs_max_sales_year = 0 

""").show(10,False)

+----------------+------------+--------------+-------------------+---------------+-----------+----------------+----------------------------+----------------+----------------------------+----------------+----------------------------+----------+------------------------+--------------+------------------+---------------+-------------+----+-----------------+-----+--------+----------+-----------------+------------+-----------------+-----------------------+-------------------+-------------------------+-------------+------------+---------+----------------+---------------+------------+-----------------------+
|fiscal_year_week|fiscal_month|supplier_local|mother_company_name|brand_type_name|brand_name |h9_l1_product_no|h9_l1_product_hierarchy_name|h9_l2_product_no|h9_l2_product_hierarchy_name|h9_l3_product_no|h9_l3_product_hierarchy_name|product_id|product_name            |web_store_flag|division_type_code|comp_store_flag|region       |area|store_format_name|state|store_id|store_name|sales_qty_1

## 3. al_inv_st_wk

In [20]:
spark.sql("""
SELECT fiscal_year_week, 
       fiscal_month, 
       supplier_local, 
       mother_company_name, 
       brand_name, 
       brand_type_name, 
       h9_l1_product_no, 
       h9_l1_product_hierarchy_name, 
       h9_l2_product_no, 
       h9_l2_product_hierarchy_name, 
       h9_l3_product_no, 
       h9_l3_product_hierarchy_name, 
       product_id, 
       product_name, 
       web_store_flag, 
       Sum(sales_qty_13w_avg)               AS sales_qty_13w_avg, 
       Sum(cogs_13w_avg)              		AS cogs_13w_avg, 
       Sum(stock_on_hand_qty)               AS stock_on_hand_qty, 
       Sum(stock_on_hand_value)             AS stock_on_hand_value, 
       Sum(stock_in_transition_qty)         AS stock_in_transition_qty, 
       Sum(stock_in_transition_value)       AS stock_in_transition_value, 
       Sum(net_sales_amt)       		    AS net_sales_amt, 
       Sum(gross_margin)    				AS gross_margin, 
       Sum(sales_qty)                       AS sales_qty, 
       Sum(net_sales_amt_ly)      		    AS net_sales_amt_ly, 
       Sum(gross_margin_ly) 				AS gross_margin_ly, 
       Sum(sales_qty_ly)                    AS sales_qty_ly
       ,current_timestamp as table_last_update
FROM   aggregates.al_inv_sku_st_wk
GROUP  BY fiscal_year_week, 
       fiscal_month, 
       supplier_local, 
       mother_company_name, 
       brand_name, 
       brand_type_name, 
       h9_l1_product_no, 
       h9_l1_product_hierarchy_name, 
       h9_l2_product_no, 
       h9_l2_product_hierarchy_name, 
       h9_l3_product_no, 
       h9_l3_product_hierarchy_name, 
       product_id, 
       product_name, 
       web_store_flag
""").show(10,False)

+----------------+------------+--------------+-------------------+--------------+---------------+----------------+----------------------------+----------------+----------------------------+----------------+----------------------------+----------+-----------------------------------------------+--------------+-----------------+------------+-----------------+-------------------+-----------------------+-------------------------+-------------+------------+---------+----------------+---------------+------------+-----------------------+
|fiscal_year_week|fiscal_month|supplier_local|mother_company_name|brand_name    |brand_type_name|h9_l1_product_no|h9_l1_product_hierarchy_name|h9_l2_product_no|h9_l2_product_hierarchy_name|h9_l3_product_no|h9_l3_product_hierarchy_name|product_id|product_name                                   |web_store_flag|sales_qty_13w_avg|cogs_13w_avg|stock_on_hand_qty|stock_on_hand_value|stock_in_transition_qty|stock_in_transition_value|net_sales_amt|gross_margin|sales_qty

## 4. al_manage_sales_wmy

In [15]:
spark.sql("""
WITH 

---------------------------------- Cross Join Hierarchy Level Data / Date / Store to Create Main Table with different granularity -------------------------------------------------------------------------------------------- 

full_cate_date(
                SELECT 
                bu_code
                , fiscal_year
                , fiscal_year_month
                , fiscal_year_week
                , vs_max_sales_year
                , NVL(substring(ext_h9_l1_hierarchy_code,3,2),'All') AS h9_l1_product_no
                , NVL(ext_h9_l1_hierarchy_name,'All') AS h9_l1_product_hierarchy_name
                , NVL(substring(ext_h9_l2_hierarchy_code,3,2),'All') AS h9_l2_product_no
                , NVL(ext_h9_l2_hierarchy_name,'All') AS h9_l2_product_hierarchy_name
                , brand_type_name
                , comp_store_flag
                , web_store_flag
                , s.Region
                , s.Area
                , store_format_name
                , CAST(store_latitude AS decimal(9,6)) AS store_latitude
                , CAST(store_longitude AS decimal(9,6)) AS store_longitude
                , CASE 
                    WHEN GROUPING_ID(bu_code, fiscal_year, fiscal_year_month, fiscal_year_week, vs_max_sales_year, comp_store_flag, web_store_flag, Region, s.Area, store_format_name, store_latitude, store_longitude
                        , ext_h9_l1_hierarchy_code, ext_h9_l1_hierarchy_name, ext_h9_l2_hierarchy_code, ext_h9_l2_hierarchy_name, brand_type_name) = 0 THEN 'with_brand'
                    WHEN GROUPING_ID(bu_code, fiscal_year, fiscal_year_month, fiscal_year_week, vs_max_sales_year, comp_store_flag, web_store_flag, Region, s.Area, store_format_name, store_latitude, store_longitude
                        , ext_h9_l1_hierarchy_code, ext_h9_l1_hierarchy_name, ext_h9_l2_hierarchy_code, ext_h9_l2_hierarchy_name, brand_type_name) = 31 THEN 'without_brand'
                    WHEN GROUPING_ID(bu_code, fiscal_year, fiscal_year_month, fiscal_year_week, vs_max_sales_year, comp_store_flag, web_store_flag, Region, s.Area, store_format_name, store_latitude, store_longitude
                        , ext_h9_l1_hierarchy_code, ext_h9_l1_hierarchy_name, ext_h9_l2_hierarchy_code, ext_h9_l2_hierarchy_name, brand_type_name) = 4071 THEN 'cate_level'
                    WHEN GROUPING_ID(bu_code, fiscal_year, fiscal_year_month, fiscal_year_week, vs_max_sales_year, comp_store_flag, web_store_flag, Region, s.Area, store_format_name, store_latitude, store_longitude
                        , ext_h9_l1_hierarchy_code, ext_h9_l1_hierarchy_name, ext_h9_l2_hierarchy_code, ext_h9_l2_hierarchy_name, brand_type_name) = 4065 THEN 'dept_level'
                    WHEN GROUPING_ID(bu_code, fiscal_year, fiscal_year_month, fiscal_year_week, vs_max_sales_year, comp_store_flag, web_store_flag, Region, s.Area, store_format_name, store_latitude, store_longitude
                        , ext_h9_l1_hierarchy_code, ext_h9_l1_hierarchy_name, ext_h9_l2_hierarchy_code, ext_h9_l2_hierarchy_name, brand_type_name) = 4095 THEN 'week_level'
                    ELSE 'Check' End AS grouping_category
                , GROUPING_ID(bu_code, fiscal_year, fiscal_year_month, fiscal_year_week, vs_max_sales_year, comp_store_flag, web_store_flag, Region, s.Area, store_format_name, store_latitude, store_longitude
                    , ext_h9_l1_hierarchy_code, ext_h9_l1_hierarchy_name, ext_h9_l2_hierarchy_code, ext_h9_l2_hierarchy_name, brand_type_name) AS grouping_id

                FROM (SELECT
                      DISTINCT 
                      fiscal_year
                      , fiscal_year_month
                      , fiscal_year_week
                      , vs_max_sales_year
                      FROM aggregates.bl_business_date
                      WHERE 0=0
                      AND vs_max_sales_year >= -1
                      AND vs_max_sales_year < 1 
                      AND max_sales_date >= fiscal_week_end_date)

                CROSS JOIN (SELECT
                            DISTINCT 
                            brand_type_name
                            , ext_h9_l1_hierarchy_code
                            , ext_h9_l1_hierarchy_name
                            , ext_h9_l2_hierarchy_code
                            , ext_h9_l2_hierarchy_name
                            FROM aggregates.product
                            WHERE 0=0
                            AND ext_h9_l1_hierarchy_code IS NOT NULL)

                CROSS JOIN (SELECT bu_key AS bu_code FROM rdm.d_business_unit)

                CROSS JOIN (SELECT
                            DISTINCT comp_store_flag
                            , web_store_flag
                            , h1_l2_hierarchy_name as Region
                            , h1_l5_hierarchy_name as Area
                            , store_format_name
                            FROM aggregates.store
                            )s

                LEFT JOIN (SELECT 
                            district_name AS area
                            , percentile_approx(latitude, 0.5,100) * 1 AS store_latitude
                            , percentile_approx(longitude, 0.5, 100) * 1 AS store_longitude
                            FROM aggregates.store
                            WHERE district_name IS NOT NULL
                            GROUP BY district_name) latlong ON NVL(s.area,'NA') = NVL(latlong.area,'NA')

                GROUP BY 
                bu_code
                , fiscal_year
                , fiscal_year_month
                , fiscal_year_week
                , vs_max_sales_year
                , comp_store_flag
                , web_store_flag
                , Region, s.Area
                , store_format_name
                , store_latitude
                , store_longitude
                , ext_h9_l1_hierarchy_code
                , ext_h9_l1_hierarchy_name
                , ext_h9_l2_hierarchy_code
                , ext_h9_l2_hierarchy_name
                , brand_type_name

                GROUPING SETS(
                (bu_code, fiscal_year, fiscal_year_month, fiscal_year_week, vs_max_sales_year)
                , (bu_code, fiscal_year, fiscal_year_month, fiscal_year_week, vs_max_sales_year, ext_h9_l1_hierarchy_code, ext_h9_l1_hierarchy_name)
                , (bu_code, fiscal_year, fiscal_year_month, fiscal_year_week, vs_max_sales_year, ext_h9_l1_hierarchy_code, ext_h9_l1_hierarchy_name, ext_h9_l2_hierarchy_code, ext_h9_l2_hierarchy_name)
                , (bu_code, fiscal_year, fiscal_year_month, fiscal_year_week, vs_max_sales_year, comp_store_flag, web_store_flag, Region, s.Area, store_format_name, store_latitude, store_longitude
                    , ext_h9_l1_hierarchy_code, ext_h9_l1_hierarchy_name, ext_h9_l2_hierarchy_code, ext_h9_l2_hierarchy_name, brand_type_name)
                , (bu_code, fiscal_year, fiscal_year_month, fiscal_year_week, vs_max_sales_year, comp_store_flag, web_store_flag, Region, s.Area, store_format_name, store_latitude, store_longitude)
                ))
                                            
---------------------------------- TXN_D Get Data with different granularity -------------------------------------------------------------------------------------------- 

, txn(
            SELECT
            txn.bu_key AS bu_code
            , d.fiscal_year
            , d.fiscal_year_month
            , d.fiscal_year_week
            , d.vs_max_sales_year
            , NVL(substring(ext_h9_l1_hierarchy_code,3,2),'All') AS h9_l1_product_no
            , p.ext_h9_l1_hierarchy_name AS h9_l1_product_hierarchy_name
            , NVL(substring(p.ext_h9_l2_hierarchy_code,3,2),'All') AS h9_l2_product_no
            , p.ext_h9_l2_hierarchy_name AS h9_l2_product_hierarchy_name
            , count(DISTINCT transaction_key) AS transaction_count
            , SUM(item_total_regular_unit_price) AS txn_gross_sales
            , sum(quantity) AS sales_qty
            , CASE 
                WHEN p.ext_h9_l1_hierarchy_code IS NULL AND p.ext_h9_l2_hierarchy_code IS NULL THEN 'week_level'
                WHEN p.ext_h9_l1_hierarchy_code IS NOT NULL AND p.ext_h9_l2_hierarchy_code IS NULL THEN 'cate_level'
                WHEN p.ext_h9_l1_hierarchy_code IS NOT NULL AND p.ext_h9_l2_hierarchy_code IS NOT NULL THEN 'dept_level'
                ELSE 'Check' END AS grouping_category
            , grouping_id(txn.bu_key, d.fiscal_year, d.fiscal_year_month, d.fiscal_year_week, d.vs_max_sales_year, p.ext_h9_l1_hierarchy_code, p.ext_h9_l1_hierarchy_name, p.ext_h9_l2_hierarchy_code, p.ext_h9_l2_hierarchy_name) AS grouping_id

            FROM rdm.f_transaction_detail txn
            LEFT JOIN aggregates.bl_business_date d ON txn.business_date_key = d.fiscal_date_key
            LEFT JOIN aggregates.product p ON txn.product_key = p.product_key

            WHERE 0=0
            AND vs_max_sales_year >= -1
            AND vs_max_sales_year < 1
            AND p.kpi_exclusion_flag = 'N'
            AND p.ext_h9_l1_hierarchy_name IS NOT NULL
            --AND fiscal_year_week IN (201836, 201936)

            GROUP BY txn.bu_key
            , d.fiscal_year
            , d.fiscal_year_month
            , d.fiscal_year_week
            , d.vs_max_sales_year
            , p.ext_h9_l1_hierarchy_code
            , p.ext_h9_l1_hierarchy_name
            , p.ext_h9_l2_hierarchy_code
            , p.ext_h9_l2_hierarchy_name

            GROUPING SETS(
            (txn.bu_key, d.fiscal_year, d.fiscal_year_month, d.fiscal_year_week, d.vs_max_sales_year)
            , (txn.bu_key, d.fiscal_year, d.fiscal_year_month, d.fiscal_year_week, d.vs_max_sales_year, p.ext_h9_l1_hierarchy_code, p.ext_h9_l1_hierarchy_name)
            , (txn.bu_key, d.fiscal_year, d.fiscal_year_month, d.fiscal_year_week, d.vs_max_sales_year, p.ext_h9_l1_hierarchy_code, p.ext_h9_l1_hierarchy_name, p.ext_h9_l2_hierarchy_code, p.ext_h9_l2_hierarchy_name)
            ))

---------------------------------- TXN_D Calculate WTD MTD YTD -------------------------------------------------------------------------------------------- 

SELECT
bu_code
, fiscal_year
-- , fiscal_year_month
, fiscal_year_week
, transaction_count_product.metric_name AS time_period
, '' AS comp_store_flag
, '' AS web_store_flag
, '' AS region
, '' AS area
, '' AS store_format_name
, CAST('' AS decimal(9,6)) AS store_latitude
, CAST('' AS decimal(9,6)) AS store_longitude
, h9_l1_product_no
, CASE WHEN h9_l1_product_hierarchy_name = 'All' THEN '' ELSE h9_l1_product_hierarchy_name END AS h9_l1_product_hierarchy_name
, h9_l2_product_no
, CASE WHEN h9_l2_product_hierarchy_name = 'All' THEN '' ELSE h9_l2_product_hierarchy_name END AS h9_l2_product_hierarchy_name
, '' AS brand_type_name

, 0 AS net_sales_amt
, 0 AS gross_profit
, 0 AS compensation
, 0 AS gross_margin
, 0 AS sales_qty
, 0 AS net_sales_amt_ly
, 0 AS gross_profit_ly
, 0 AS compensation_ly
, 0 AS gross_margin_ly
, 0 AS sales_qty_ly
, 0 AS people_count_in
, 0 AS transaction_count_store
, 0 AS txn_gross_sales_store
, 0 AS txn_sales_qty_store
, 0 AS people_count_in_ly
, 0 AS transaction_count_store_ly
, 0 AS txn_gross_sales_store_ly
, 0 AS txn_sales_qty_store_ly

, transaction_count_product.metric_value AS transaction_count_product
, txn_gross_sales_product.metric_value AS txn_gross_sales_product
, txn_sales_qty_product.metric_value AS txn_sales_qty_product
, transaction_count_product_ly.metric_value AS transaction_count_product_ly
, txn_gross_sales_product_ly.metric_value AS txn_gross_sales_product_ly
, txn_sales_qty_product_ly.metric_value AS txn_sales_qty_product_ly

, current_timestamp AS table_last_update

FROM (
        SELECT

        full_cate_date.bu_code
        , full_cate_date.fiscal_year
        , full_cate_date.fiscal_year_month
        , full_cate_date.fiscal_year_week
        , full_cate_date.vs_max_sales_year
        , CASE WHEN full_cate_date.h9_l1_product_no  IS NULL THEN 'All' ELSE full_cate_date.h9_l1_product_no  END AS h9_l1_product_no 
        , full_cate_date.h9_l1_product_hierarchy_name  AS h9_l1_product_hierarchy_name 
        , CASE WHEN full_cate_date.h9_l2_product_no  IS NULL THEN 'All' ELSE full_cate_date.h9_l2_product_no  END AS h9_l2_product_no 
        , full_cate_date.h9_l2_product_hierarchy_name

        , NVL(txn.transaction_count,0) AS transaction_count_product_WTD
        , NVL(SUM(txn.transaction_count) OVER mtd,0) AS transaction_count_product_MTD
        , NVL(SUM(txn.transaction_count) OVER ytd,0) AS transaction_count_product_YTD

        , NVL(txn.txn_gross_sales,0) AS txn_gross_sales_product_WTD
        , NVL(SUM(txn.txn_gross_sales) OVER mtd,0) AS txn_gross_sales_product_MTD
        , NVL(SUM(txn.txn_gross_sales) OVER ytd,0) AS txn_gross_sales_product_YTD

        , NVL(txn.sales_qty,0) AS txn_sales_qty_product_WTD
        , NVL(SUM(txn.sales_qty) OVER mtd,0) AS txn_sales_qty_product_MTD
        , NVL(SUM(txn.sales_qty) OVER ytd,0) AS txn_sales_qty_product_YTD

        , NVL(txn_ly.transaction_count_ly,0) AS transaction_count_product_ly_WTD
        , NVL(SUM(txn_ly.transaction_count_ly) OVER mtd,0) AS transaction_count_product_ly_MTD
        , NVL(SUM(txn_ly.transaction_count_ly) OVER ytd,0) AS transaction_count_product_ly_YTD

        , NVL(txn_ly.txn_gross_sales_ly,0) AS txn_gross_sales_product_ly_WTD
        , NVL(SUM(txn_ly.txn_gross_sales_ly) OVER mtd,0) AS txn_gross_sales_product_ly_MTD
        , NVL(SUM(txn_ly.txn_gross_sales_ly) OVER ytd,0) AS txn_gross_sales_product_ly_YTD

        , NVL(txn_ly.sales_qty_ly,0) AS txn_sales_qty_product_ly_WTD
        , NVL(SUM(txn_ly.sales_qty_ly) OVER mtd,0) AS txn_sales_qty_product_ly_MTD
        , NVL(SUM(txn_ly.sales_qty_ly) OVER ytd,0) AS txn_sales_qty_product_ly_YTD

        FROM full_cate_date
        LEFT JOIN txn ON full_cate_date.fiscal_year_week = txn.fiscal_year_week
                      AND full_cate_date.h9_l1_product_no  = nvl(txn.h9_l1_product_no,'All') 
                      AND full_cate_date.h9_l2_product_no  = nvl(txn.h9_l2_product_no,'All') 
                      AND full_cate_date.grouping_category = txn.grouping_category
                      
        LEFT JOIN (SELECT
                        fiscal_year AS fiscal_year_ly
                        , fiscal_year_month AS fiscal_year_month_ly
                        , fiscal_year_week AS fiscal_year_week_ly
                        , h9_l1_product_no  AS h9_l1_product_no_ly
                        , h9_l1_product_hierarchy_name AS h9_l1_product_hierarchy_name_ly
                        , h9_l2_product_no AS h9_l2_product_no_ly
                        , h9_l2_product_hierarchy_name AS h9_l2_product_hierarchy_name_ly
                        , grouping_id AS grouping_id_ly
                        , grouping_category AS grouping_category_ly
                        , transaction_count AS transaction_count_ly
                        , txn_gross_sales AS txn_gross_sales_ly
                        , sales_qty AS sales_qty_ly
                    FROM txn) txn_ly ON full_cate_date.fiscal_year_week - 100 = txn_ly.fiscal_year_week_ly
                                     AND full_cate_date.h9_l1_product_no = nvl(txn_ly.h9_l1_product_no_ly,'All') 
                                     AND full_cate_date.h9_l2_product_no = nvl(txn_ly.h9_l2_product_no_ly,'All') 
                                     AND full_cate_date.grouping_category = txn_ly.grouping_category_ly
        
        WHERE 0=0
        AND full_cate_date.grouping_category IN ('week_level', 'dept_level', 'cate_level')

        WINDOW 
        mtd AS (PARTITION BY 
                 full_cate_date.fiscal_year_month, full_cate_date.h9_l1_product_no, full_cate_date.h9_l1_product_hierarchy_name, full_cate_date.h9_l2_product_no, full_cate_date.h9_l2_product_hierarchy_name
                 ORDER BY full_cate_date.fiscal_year_week, full_cate_date.h9_l1_product_no, full_cate_date.h9_l1_product_hierarchy_name, full_cate_date.h9_l2_product_no, full_cate_date.h9_l2_product_hierarchy_name
                )
        , ytd AS (PARTITION BY 
                 full_cate_date.fiscal_year, full_cate_date.h9_l1_product_no, full_cate_date.h9_l1_product_hierarchy_name, full_cate_date.h9_l2_product_no, full_cate_date.h9_l2_product_hierarchy_name
                 ORDER BY full_cate_date.fiscal_year_week, full_cate_date.h9_l1_product_no, full_cate_date.h9_l1_product_hierarchy_name, full_cate_date.h9_l2_product_no, full_cate_date.h9_l2_product_hierarchy_name
                ))

---------------------------------- TXN_D Transpose relevant columns -------------------------------------------------------------------------------------------- 


LATERAL VIEW EXPLODE (
                        MAP("WTD",transaction_count_product_WTD
                            , "MTD", transaction_count_product_MTD
                            , "YTD", transaction_count_product_YTD
                      ))transaction_count_product AS metric_name, metric_value

LATERAL VIEW EXPLODE (
                        MAP("WTD",txn_gross_sales_product_WTD
                            , "MTD", txn_gross_sales_product_MTD
                            , "YTD", txn_gross_sales_product_YTD
                      ))txn_gross_sales_product AS metric_name, metric_value
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD",txn_sales_qty_product_WTD
                            , "MTD", txn_sales_qty_product_MTD
                            , "YTD", txn_sales_qty_product_YTD
                      ))txn_sales_qty_product AS metric_name, metric_value
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD",transaction_count_product_ly_WTD
                            , "MTD", transaction_count_product_ly_MTD
                            , "YTD", transaction_count_product_ly_YTD
                      ))transaction_count_product_ly AS metric_name, metric_value
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD",txn_gross_sales_product_ly_WTD
                            , "MTD", txn_gross_sales_product_ly_MTD
                            , "YTD", txn_gross_sales_product_ly_YTD
                      ))txn_gross_sales_product_ly AS metric_name, metric_value
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD", txn_sales_qty_product_ly_WTD
                            , "MTD", txn_sales_qty_product_ly_MTD
                            , "YTD", txn_sales_qty_product_ly_YTD
                      ))txn_sales_qty_product_ly AS metric_name, metric_value

WHERE 0=0
AND transaction_count_product.metric_name = txn_gross_sales_product.metric_name
AND transaction_count_product.metric_name = txn_sales_qty_product.metric_name
AND transaction_count_product.metric_name = transaction_count_product_ly.metric_name
AND transaction_count_product.metric_name = txn_gross_sales_product_ly.metric_name
AND transaction_count_product.metric_name = txn_sales_qty_product_ly.metric_name

AND (transaction_count_product.metric_value + txn_gross_sales_product.metric_value + txn_sales_qty_product.metric_value + transaction_count_product_ly.metric_value + txn_gross_sales_product_ly.metric_value + txn_sales_qty_product_ly.metric_value) <> 0

AND NVL(vs_max_sales_year,'NA') = 0

---------------------------------- H9L2  -------------------------------------------------------------------------------------------- 

UNION ALL(


SELECT
bu_code
, fiscal_year
, fiscal_year_week
, net_sales_amt.metric_name AS time_period
, comp_store_flag
, web_store_flag
, region
, area
, store_format_name
, store_latitude
, store_longitude
, h9_l1_product_no
, CASE WHEN h9_l1_product_hierarchy_name = 'All' THEN '' ELSE h9_l1_product_hierarchy_name END AS h9_l1_product_hierarchy_name
, h9_l2_product_no
, CASE WHEN h9_l2_product_hierarchy_name = 'All' THEN '' ELSE h9_l2_product_hierarchy_name END AS h9_l2_product_hierarchy_name
, brand_type_name

, NVL(net_sales_amt.metric_value,0) AS net_sales_amt
, NVL(gross_profit.metric_value,0) AS gross_profit
, NVL(compensation.metric_value,0) AS compensation
, NVL(gross_margin.metric_value,0) AS gross_margin
, NVL(sales_qty.metric_value,0) AS sales_qty

, NVL(net_sales_amt_ly.metric_value,0) AS net_sales_amt_ly
, NVL(gross_profit_ly.metric_value,0) AS gross_profit_ly
, NVL(compensation_ly.metric_value,0) AS compensation_ly
, NVL(gross_margin_ly.metric_value,0) AS gross_margin_ly
, NVL(sales_qty_ly.metric_value,0) AS sales_qty_ly

, 0 AS people_count_in
, 0 AS transaction_count_store
, 0 AS txn_gross_sales_store
, 0 AS txn_sales_qty_store
, 0 AS people_count_in_ly
, 0 AS transaction_count_store_ly
, 0 AS txn_gross_sales_store_ly
, 0 AS txn_sales_qty_store_ly
, 0 AS transaction_count_product
, 0 AS txn_gross_sales_product
, 0 AS txn_sales_qty_product
, 0 AS transaction_count_product_ly
, 0 AS txn_gross_sales_product_ly
, 0 AS txn_sales_qty_product_ly
, current_timestamp AS table_last_update

---------------------------------- H9L2 Calculate WTD MTD YTD -------------------------------------------------------------------------------------------- 

FROM(
        SELECT
        full_cate_date.bu_code
        , full_cate_date.fiscal_year
        , full_cate_date.fiscal_year_month
        , full_cate_date.fiscal_year_week
        , full_cate_date.comp_store_flag
        , full_cate_date.web_store_flag
        , full_cate_date.region
        , full_cate_date.area
        , full_cate_date.store_format_name
        , full_cate_date.store_latitude
        , full_cate_date.store_longitude
        , full_cate_date.h9_l1_product_no
        , full_cate_date.h9_l1_product_hierarchy_name
        , full_cate_date.h9_l2_product_no
        , full_cate_date.h9_l2_product_hierarchy_name
        , full_cate_date.brand_type_name

        , NVL(net_sales_amt,0) AS net_sales_amt_WTD
        , NVL(SUM(net_sales_amt) OVER mtd,0) AS net_sales_amt_MTD
        , NVL(SUM(net_sales_amt) OVER ytd,0) AS net_sales_amt_YTD

        , NVL(gross_profit,0) AS gross_profit_WTD
        , NVL(SUM(gross_profit) OVER mtd,0)  AS gross_profit_MTD
        , NVL(SUM(gross_profit) OVER ytd,0)  AS gross_profit_YTD

        , NVL(compensation,0) AS compensation_WTD
        , NVL(SUM(compensation) OVER mtd,0) AS compensation_MTD
        , NVL(SUM(compensation) OVER ytd,0) AS compensation_YTD

        , NVL(gross_margin,0) AS gross_margin_WTD
        , NVL(SUM(gross_margin) OVER mtd,0) AS gross_margin_MTD
        , NVL(SUM(gross_margin) OVER ytd,0) AS gross_margin_YTD

        , NVL(net_sales_amt_ly,0) AS net_sales_amt_ly_WTD
        , NVL(SUM(net_sales_amt_ly) OVER mtd,0) AS net_sales_amt_ly_MTD
        , NVL(SUM(net_sales_amt_ly) OVER ytd,0) AS net_sales_amt_ly_YTD

        , NVL(gross_profit_ly,0) AS gross_profit_ly_WTD
        , NVL(SUM(gross_profit_ly) OVER mtd,0) AS gross_profit_ly_MTD
        , NVL(SUM(gross_profit_ly) OVER ytd,0) AS gross_profit_ly_YTD 

        , NVL(compensation_ly,0) AS compensation_ly_WTD
        , NVL(SUM(compensation_ly) OVER mtd,0) AS compensation_ly_MTD
        , NVL(SUM(compensation_ly) OVER ytd,0) AS compensation_ly_YTD 

        , NVL(gross_margin_ly,0) AS gross_margin_ly_WTD
        , NVL(SUM(gross_margin_ly) OVER mtd,0) AS gross_margin_ly_MTD
        , NVL(SUM(gross_margin_ly) OVER ytd,0) AS gross_margin_ly_YTD  

        , NVL(sales_qty,0) AS sales_qty_WTD
        , NVL(SUM(sales_qty) OVER mtd,0) AS sales_qty_MTD
        , NVL(SUM(sales_qty) OVER ytd,0) AS sales_qty_YTD

        , NVL(sales_qty_ly,0) AS sales_qty_ly_WTD
        , NVL(SUM(sales_qty_ly) OVER mtd,0) AS sales_qty_ly_MTD
        , NVL(SUM(sales_qty_ly) OVER ytd,0) AS sales_qty_ly_YTD



        FROM full_cate_date
        LEFT JOIN (SELECT
                  fiscal_year_week
                  , comp_store_flag
                  , web_store_flag
                  , region
                  , area
                  , store_format_name
                  , h9_l1_product_no
                  , h9_l2_product_no
                  , brand_type_name
                  , SUM(net_sales_amt) AS net_sales_amt
                  , SUM(gross_profit) AS gross_profit
                  , SUM(compensation) AS compensation
                  , SUM(gross_margin) AS gross_margin
                  , SUM(sales_qty) AS sales_qty
                  , SUM(net_sales_amt_ly) AS net_sales_amt_ly
                  , SUM(gross_profit_ly) AS gross_profit_ly
                  , SUM(compensation_ly) AS compensation_ly
                  , SUM(gross_margin_ly) AS gross_margin_ly
                  , SUM(sales_qty_ly) AS sales_qty_ly
                   FROM aggregates.bl_sales_brd_h9l2_st_sup_wk 
                   WHERE sku_flag = 1
                   GROUP BY
                   fiscal_year_week
                  , comp_store_flag
                  , web_store_flag
                  , region
                  , area
                  , store_format_name
                  , h9_l1_product_no
                  , h9_l2_product_no
                  , brand_type_name )  h9l2 ON NVL(h9l2.fiscal_year_week,'NA') = NVL(full_cate_date.fiscal_year_week,'NA')
                                            AND NVL(h9l2.comp_store_flag,'NA') = NVL(full_cate_date.comp_store_flag,'NA')
                                            AND NVL(h9l2.web_store_flag,'NA') = NVL(full_cate_date.web_store_flag,'NA')
                                            AND NVL(h9l2.region,'NA') = NVL(full_cate_date.region,'NA')
                                            AND NVL(h9l2.area,'NA') = NVL(full_cate_date.area,'NA')
                                            AND NVL(h9l2.store_format_name,'NA') = NVL(full_cate_date.store_format_name,'NA')
                                            AND NVL(h9l2.h9_l1_product_no,'NA') = NVL(full_cate_date.h9_l1_product_no,'NA')
                                            AND NVL(h9l2.h9_l2_product_no,'NA') = NVL(full_cate_date.h9_l2_product_no,'NA')
                                            AND NVL(h9l2.brand_type_name,'NA') = NVL(full_cate_date.brand_type_name,'NA')



        WHERE 0=0
        AND full_cate_date.grouping_category = 'with_brand'
        AND NVL(full_cate_date.vs_max_sales_year,'NA') = 0


        WINDOW 
         mtd AS (PARTITION BY 
                 full_cate_date.fiscal_year_month, full_cate_date.comp_store_flag, full_cate_date.web_store_flag, full_cate_date.region, full_cate_date.area, full_cate_date.store_format_name, full_cate_date.store_latitude
                 , full_cate_date.store_longitude, full_cate_date.h9_l1_product_no , full_cate_date.h9_l1_product_hierarchy_name, full_cate_date.h9_l2_product_no, full_cate_date.h9_l2_product_hierarchy_name, full_cate_date.brand_type_name  
                 ORDER BY full_cate_date.fiscal_year_week, full_cate_date.comp_store_flag, full_cate_date.web_store_flag, full_cate_date.region, full_cate_date.area , full_cate_date.store_format_name
                 , full_cate_date.store_latitude, full_cate_date.store_longitude, full_cate_date.h9_l1_product_no , full_cate_date.h9_l1_product_hierarchy_name, full_cate_date.h9_l2_product_no, full_cate_date.h9_l2_product_hierarchy_name, full_cate_date.brand_type_name 
                )
        , ytd AS (PARTITION BY 
                 full_cate_date.fiscal_year, full_cate_date.comp_store_flag, full_cate_date.web_store_flag, full_cate_date.region, full_cate_date.area, full_cate_date.store_format_name, full_cate_date.store_latitude
                 , full_cate_date.store_longitude, full_cate_date.h9_l1_product_no , full_cate_date.h9_l1_product_hierarchy_name, full_cate_date.h9_l2_product_no, full_cate_date.h9_l2_product_hierarchy_name, full_cate_date.brand_type_name  
                 ORDER BY full_cate_date.fiscal_year_week, full_cate_date.comp_store_flag, full_cate_date.web_store_flag, full_cate_date.region, full_cate_date.area , full_cate_date.store_format_name
                 , full_cate_date.store_latitude, full_cate_date.store_longitude, full_cate_date.h9_l1_product_no , full_cate_date.h9_l1_product_hierarchy_name, full_cate_date.h9_l2_product_no, full_cate_date.h9_l2_product_hierarchy_name, full_cate_date.brand_type_name))

---------------------------------- h9l2 Transpose relevant columns -------------------------------------------------------------------------------------------- 

LATERAL VIEW EXPLODE (
                        MAP("WTD",net_sales_amt_WTD
                            , "MTD", net_sales_amt_MTD
                            , "YTD", net_sales_amt_YTD
                      ))net_sales_amt AS metric_name, metric_value
LATERAL VIEW EXPLODE (
                        MAP("WTD",net_sales_amt_ly_WTD
                            , "MTD", net_sales_amt_ly_MTD
                            , "YTD", net_sales_amt_ly_YTD
                      ))net_sales_amt_ly AS metric_name, metric_value
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD",gross_profit_WTD
                            , "MTD", gross_profit_MTD
                            , "YTD", gross_profit_YTD
                      ))gross_profit AS metric_name, metric_value
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD",gross_profit_ly_WTD
                            , "MTD", gross_profit_ly_MTD
                            , "YTD", gross_profit_ly_YTD
                      ))gross_profit_ly AS metric_name, metric_value
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD",compensation_WTD
                            , "MTD", compensation_MTD
                            , "YTD", compensation_YTD
                      ))compensation AS metric_name, metric_value 
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD",compensation_ly_WTD
                            , "MTD", compensation_ly_MTD
                            , "YTD", compensation_ly_YTD
                      ))compensation_ly AS metric_name, metric_value
                     
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD",gross_margin_WTD
                            , "MTD", gross_margin_MTD
                            , "YTD", gross_margin_YTD
                      ))gross_margin AS metric_name, metric_value
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD",gross_margin_ly_WTD
                            , "MTD", gross_margin_ly_MTD
                            , "YTD", gross_margin_ly_YTD
                      ))gross_margin_ly AS metric_name, metric_value   
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD",sales_qty_WTD
                            , "MTD", sales_qty_MTD
                            , "YTD", sales_qty_YTD
                      ))sales_qty AS metric_name, metric_value
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD",sales_qty_ly_WTD
                            , "MTD", sales_qty_ly_MTD
                            , "YTD", sales_qty_ly_YTD
                      ))sales_qty_ly AS metric_name, metric_value  
                      
      
WHERE 0=0
AND net_sales_amt.metric_name = net_sales_amt_ly.metric_name
AND net_sales_amt.metric_name = gross_profit.metric_name
AND net_sales_amt.metric_name = gross_profit_ly.metric_name
AND net_sales_amt.metric_name = compensation.metric_name
AND net_sales_amt.metric_name = compensation_ly.metric_name
AND net_sales_amt.metric_name = gross_margin.metric_name
AND net_sales_amt.metric_name = gross_margin_ly.metric_name
AND net_sales_amt.metric_name = sales_qty.metric_name
AND net_sales_amt.metric_name = sales_qty_ly.metric_name

AND (net_sales_amt.metric_value + gross_profit.metric_value +  compensation.metric_value + gross_margin.metric_value + sales_qty.metric_value 
+ net_sales_amt_ly.metric_value + gross_profit_ly.metric_value +  compensation_ly.metric_value + gross_margin_ly.metric_value + sales_qty_ly.metric_value) <> 0

)

---------------------------------- Traffic -------------------------------------------------------------------------------------------- 

UNION ALL (

SELECT
bu_code
, fiscal_year
, fiscal_year_week
, people_count_in.metric_name AS time_period
, comp_store_flag
, web_store_flag
, region
, area
, store_format_name
, store_latitude
, store_longitude
, CASE WHEN h9_l1_product_no = 'All' THEN '' ELSE h9_l1_product_no END AS h9_l1_product_no
, CASE WHEN h9_l1_product_hierarchy_name = 'All' THEN '' ELSE h9_l1_product_hierarchy_name END AS h9_l1_product_hierarchy_name
, CASE WHEN h9_l2_product_no = 'All' THEN '' ELSE h9_l2_product_no END AS h9_l2_product_no
, CASE WHEN h9_l2_product_hierarchy_name = 'All' THEN '' ELSE h9_l2_product_hierarchy_name END AS h9_l2_product_hierarchy_name

, brand_type_name


, 0 AS net_sales_amt
, 0 AS gross_profit
, 0 AS compensation
, 0 AS gross_margin
, 0 AS sales_qty

, 0 AS net_sales_amt_ly
, 0 AS gross_profit_ly
, 0 AS compensation_ly
, 0 AS gross_margin_ly
, 0 AS sales_qty_ly

, NVL(people_count_in.metric_value,0) AS people_count_in
, NVL(transaction_count.metric_value,0) AS transaction_count_store
, NVL(txn_gross_sales.metric_value,0) AS txn_gross_sales_store
, NVL(txn_sales_qty.metric_value,0) AS txn_sales_qty_store
, NVL(people_count_in_ly.metric_value,0) AS people_count_in_ly
, NVL(transaction_count_ly.metric_value,0) AS transaction_count_store_ly
, NVL(txn_gross_sales.metric_value,0) AS txn_gross_sales_store_ly
, NVL(txn_sales_qty_ly.metric_value,0) AS txn_sales_qty_store_ly
, 0 AS transaction_count_product
, 0 AS txn_gross_sales_product
, 0 AS txn_sales_qty_product
, 0 AS transaction_count_product_ly
, 0 AS txn_gross_sales_product_ly
, 0 AS txn_sales_qty_product_ly
, current_timestamp AS table_last_update

---------------------------------- Traffic Calculate WTD MTD YTD -------------------------------------------------------------------------------------------- 

FROM(
    SELECT
    full_cate_date.bu_code
    , full_cate_date.fiscal_year
    , full_cate_date.fiscal_year_month
    , full_cate_date.fiscal_year_week
    , full_cate_date.comp_store_flag
    , full_cate_date.web_store_flag
    , full_cate_date.region
    , full_cate_date.area
    , full_cate_date.store_format_name
    , full_cate_date.store_latitude
    , full_cate_date.store_longitude
    , full_cate_date.h9_l1_product_no
    , full_cate_date.h9_l1_product_hierarchy_name
    , full_cate_date.h9_l2_product_no
    , full_cate_date.h9_l2_product_hierarchy_name
    , full_cate_date.brand_type_name

    , NVL(people_count_in,0) as people_count_in_WTD
    , NVL(SUM(people_count_in) OVER mtd,0) as people_count_in_MTD
    , NVL(SUM(people_count_in) OVER ytd,0) as people_count_in_YTD

    , NVL(transaction_count,0) as transaction_count_WTD
    , NVL(SUM(transaction_count) OVER mtd,0) as transaction_count_MTD
    , NVL(SUM(transaction_count) OVER ytd,0) as transaction_count_YTD

    , NVL(txn_gross_sales,0) as txn_gross_sales_WTD
    , NVL(SUM(txn_gross_sales) OVER mtd,0) as txn_gross_sales_MTD
    , NVL(SUM(txn_gross_sales) OVER ytd,0) as txn_gross_sales_YTD

    , NVL(txn_sales_qty,0) as txn_sales_qty_WTD
    , NVL(SUM(txn_sales_qty) OVER mtd,0) as txn_sales_qty_MTD
    , NVL(SUM(txn_sales_qty) OVER ytd,0) as txn_sales_qty_YTD

    , NVL(people_count_in_ly,0) as people_count_in_ly_WTD
    , NVL(SUM(people_count_in_ly) OVER mtd,0) as people_count_in_ly_MTD
    , NVL(SUM(people_count_in_ly) OVER ytd,0) as people_count_in_ly_YTD

    , NVL(transaction_count_ly,0) as transaction_count_ly_WTD
    , NVL(SUM(transaction_count_ly) OVER mtd,0) as transaction_count_ly_MTD
    , NVL(SUM(transaction_count_ly) OVER ytd,0) as transaction_count_ly_YTD

    , NVL(txn_gross_sales_ly,0) as txn_gross_sales_ly_WTD
    , NVL(SUM(txn_gross_sales_ly) OVER mtd,0) as txn_gross_sales_ly_MTD
    , NVL(SUM(txn_gross_sales_ly) OVER ytd,0) as txn_gross_sales_ly_YTD

    , NVL(txn_sales_qty_ly,0) as txn_sales_qty_ly_WTD
    , NVL(SUM(txn_sales_qty_ly) OVER mtd,0) as txn_sales_qty_ly_MTD
    , NVL(SUM(txn_sales_qty_ly) OVER ytd,0) as txn_sales_qty_ly_YTD

    FROM full_cate_date
    LEFT JOIN ( SELECT
                fiscal_year_week
                , comp_store_flag
                , web_store_flag
                , region
                , area
                , store_format_name
                , NVL(SUM(people_count_in),0) AS people_count_in
                , NVL(SUM(transaction_count),0) AS transaction_count
                , NVL(SUM(txn_gross_sales),0) AS txn_gross_sales
                , NVL(SUM(txn_sales_qty),0) AS txn_sales_qty
                , NVL(SUM(people_count_in_ly),0) AS people_count_in_ly
                , NVL(SUM(transaction_count_ly),0) AS transaction_count_ly
                , NVL(SUM(txn_gross_sales_ly),0) AS txn_gross_sales_ly
                , NVL(SUM(txn_sales_qty_ly),0) AS txn_sales_qty_ly

                FROM aggregates.bl_traffic_st_wk

                GROUP BY fiscal_year_week
                , comp_store_flag
                , web_store_flag
                , region
                , area
                , store_format_name) traffic ON NVL(traffic.fiscal_year_week,'NA') = NVL(full_cate_date.fiscal_year_week,'NA')
                                             AND NVL(traffic.comp_store_flag,'NA') = NVL(full_cate_date.comp_store_flag,'NA')
                                             AND NVL(traffic.web_store_flag,'NA') = NVL(full_cate_date.web_store_flag,'NA')
                                             AND NVL(traffic.region,'NA') = NVL(full_cate_date.region,'NA')
                                             AND NVL(traffic.area,'NA') = NVL(full_cate_date.area,'NA')
                                             AND NVL(traffic.store_format_name,'NA') = NVL(full_cate_date.store_format_name,'NA')
                                             AND NVL(traffic.fiscal_year_week,'NA') = NVL(full_cate_date.fiscal_year_week,'NA')
                                             AND NVL(traffic.fiscal_year_week,'NA') = NVL(full_cate_date.fiscal_year_week,'NA')

    WHERE 0=0
    AND full_cate_date.grouping_category = 'without_brand'
    AND NVL(full_cate_date.vs_max_sales_year,'NA') = 0

     WINDOW 
             mtd AS (PARTITION BY 
                     full_cate_date.fiscal_year_month, full_cate_date.comp_store_flag, full_cate_date.web_store_flag, full_cate_date.region, full_cate_date.area, full_cate_date.store_format_name, full_cate_date.store_latitude
                     , full_cate_date.store_longitude
                     ORDER BY full_cate_date.fiscal_year_week, full_cate_date.comp_store_flag, full_cate_date.web_store_flag, full_cate_date.region, full_cate_date.area , full_cate_date.store_format_name
                     , full_cate_date.store_latitude, full_cate_date.store_longitude)
            , ytd AS (PARTITION BY 
                     full_cate_date.fiscal_year, full_cate_date.comp_store_flag, full_cate_date.web_store_flag, full_cate_date.region, full_cate_date.area, full_cate_date.store_format_name, full_cate_date.store_latitude
                     , full_cate_date.store_longitude 
                     ORDER BY full_cate_date.fiscal_year_week, full_cate_date.comp_store_flag, full_cate_date.web_store_flag, full_cate_date.region, full_cate_date.area , full_cate_date.store_format_name
                     , full_cate_date.store_latitude, full_cate_date.store_longitude))

---------------------------------- Traffic Transpose relevant columns -------------------------------------------------------------------------------------------- 

LATERAL VIEW EXPLODE (
                        MAP("WTD",people_count_in_WTD
                            , "MTD", people_count_in_MTD
                            , "YTD", people_count_in_YTD
                      ))people_count_in AS metric_name, metric_value

LATERAL VIEW EXPLODE (
                        MAP("WTD",transaction_count_WTD
                            , "MTD", transaction_count_MTD
                            , "YTD", transaction_count_YTD
                      ))transaction_count AS metric_name, metric_value
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD",txn_gross_sales_WTD
                            , "MTD", txn_gross_sales_MTD
                            , "YTD", txn_gross_sales_YTD
                      ))txn_gross_sales AS metric_name, metric_value
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD",txn_sales_qty_WTD
                            , "MTD", txn_sales_qty_MTD
                            , "YTD", txn_sales_qty_YTD
                      ))txn_sales_qty AS metric_name, metric_value

LATERAL VIEW EXPLODE (
                        MAP("WTD",people_count_in_ly_WTD
                            , "MTD", people_count_in_ly_MTD
                            , "YTD", people_count_in_ly_YTD
                      ))people_count_in_ly AS metric_name, metric_value

LATERAL VIEW EXPLODE (
                        MAP("WTD",transaction_count_ly_WTD
                            , "MTD", transaction_count_ly_MTD
                            , "YTD", transaction_count_ly_YTD
                      ))transaction_count_ly AS metric_name, metric_value
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD",txn_gross_sales_ly_WTD
                            , "MTD", txn_gross_sales_ly_MTD
                            , "YTD", txn_gross_sales_ly_YTD
                      ))txn_gross_sales_ly AS metric_name, metric_value
                      
LATERAL VIEW EXPLODE (
                        MAP("WTD",txn_sales_qty_ly_WTD
                            , "MTD", txn_sales_qty_ly_MTD
                            , "YTD", txn_sales_qty_ly_YTD
                      ))txn_sales_qty_ly AS metric_name, metric_value

WHERE 0=0
AND people_count_in.metric_name = transaction_count.metric_name
AND people_count_in.metric_name = txn_gross_sales.metric_name
AND people_count_in.metric_name = txn_sales_qty.metric_name
AND people_count_in.metric_name = people_count_in_ly.metric_name
AND people_count_in.metric_name = transaction_count_ly.metric_name
AND people_count_in.metric_name = txn_gross_sales_ly.metric_name
AND people_count_in.metric_name = txn_sales_qty_ly.metric_name
)

""").show(10,False)

+-------+-----------+----------------+-----------+---------------+--------------+------+----+-----------------+--------------+---------------+----------------+----------------------------+----------------+----------------------------+---------------+-------------+------------+------------+------------+---------+----------------+---------------+---------------+---------------+------------+---------------+-----------------------+---------------------+-------------------+------------------+--------------------------+------------------------+----------------------+-------------------------+-----------------------+---------------------+----------------------------+--------------------------+------------------------+-----------------------+
|bu_code|fiscal_year|fiscal_year_week|time_period|comp_store_flag|web_store_flag|region|area|store_format_name|store_latitude|store_longitude|h9_l1_product_no|h9_l1_product_hierarchy_name|h9_l2_product_no|h9_l2_product_hierarchy_name|brand_type_name|net_s

## 5. al_barcode

In [14]:
spark.sql("""
select 
nvl(bu.bu_key,'-')					as bu_code
,nvl(bu.bu_name,'-') 				as bu_name
,nvl(p.principle_supplier_code,'-') as supplier_code
,nvl(p.mother_company_name,'-') 	as supplier_name
,'-' 								as brand_code
,nvl(p.ib_brand_name,'-') 			as brand_name
,nvl(p.product_id,'-')				as product_id
,nvl(p.product_other_name,'-') 		as product_name_eng
,nvl(p.product_name,'-') 			as product_name_loc
,att.ATTRIBUTE_VALUE_STRING 		as barcode
,current_timestamp 					as table_last_update
from aggregates.product p
join rdm.d_attributes atts on atts.attributes_key = p.product_attributes_key
join rdm.d_attribute att on att.attribute_key = atts.attribute_key
left join rdm.d_business_unit bu on bu.bu_key = p.bu_key
where att.attribute_type = 'Barcode'
""").show(10,False)

+-------+----------------+-------------+------------------+----------+------------+----------+---------------------------------------------------+--------------------------------------+--------------+-----------------------+
|bu_code|bu_name         |supplier_code|supplier_name     |brand_code|brand_name  |product_id|product_name_eng                                   |product_name_loc                      |barcode       |table_last_update      |
+-------+----------------+-------------+------------------+----------+------------+----------+---------------------------------------------------+--------------------------------------+--------------+-----------------------+
|WTCTW  |Watsons - Taiwan|-            |OTHER SUPPLIER    |-         |ZUSHIANG    |100706    |ZUSHIANG ELECTRIC BLANKET ZOG-2330B                |日象暄暖微電腦溫控電蓋毯 ZOG-2330B 雙人規格(160x130公分)|4717154554490 |2019-11-15 06:45:52.144|
|WTCTW  |Watsons - Taiwan|UNI          |UNILEVER          |-         |DOVE        |100822    |Dove+V